In [ ]:
!pip install mesa
!pip install names_dataset 
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.7/177.7 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 MB 38.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 107.9 MB/s eta 0:00:00
  Created wheel for names_dataset: filename=names_dataset-3.1.0-py3-none-any.whl size=116832757 sha256=6f2cb8925394066a2eb3921685daa0ba2da11428ead4cd748c03e0a77c82f996
  Stored in directory: /root/.cache/pip/wheels/c2/e9/aa/2306c390640835aea9ead91c2deddd1a87a739ab3ee41f5ac4
Successfully built names_dataset
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━

In [ ]:
# Step 1: Mount Google Drive to write results later
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# # Step 2: Set up paths and import your custom library
# import sys
# # Assuming 'crowd' is stored in 'MyDrive', adjust the path if needed
# sys.path.append('/content/drive/My Drive/Crowd_Related_Work/mesa')

# Import mesa
import mesa
from mesa.experimental.cell_space import CellAgent, Network

In [ ]:
import json
import os
import random
import time
from names_dataset import NameDataset
import networkx as nx
from enum import Enum

In [ ]:
%cd /content/drive/My Drive/Crowd_Related_Work/mesa

/content/drive/My Drive/Crowd_Related_Work/mesa


In [ ]:
class State(Enum):
    SUSCEPTIBLE = 0
    INFECTED = 1
    RECOVERED = 2

In [ ]:
class World(mesa.Model):
  """
    This class holds the environment which we will utilize in our simulation
  """

  def __init__(
      self,
      initial_susceptible = 9,
      initial_infected = 1,
      network_degree = 5,
      llm_model = None,
      tokenizer = None
  ):
    # Step 1: Initialize the Network environment
    super().__init__()
    self.agent_count = initial_susceptible + initial_infected
    self.G = nx.random_regular_graph(network_degree, self.agent_count)
    self.grid = Network(self.G, random=self.random)

    # Model level variables that will be utilized within agents
    self.llm_model = llm_model
    self.tokenizer = tokenizer
    self.newly_infected = []
    self.responses = {}
    self.epoch = -1
    self.inference_count = 0
    self.inference_time_taken = 0

    #Initiate data collector
    self.datacollector = mesa.DataCollector(
          {"Susceptible": compute_num_susceptible,
            "Infected": compute_num_infected,
            "Recovered": compute_num_recovered,
            "# Home": compute_num_at_home,
            "# Grid":compute_num_on_grid
         }
    )

    # Create agents
    random_nodetypes = []
    random_nodetypes.extend([ State.SUSCEPTIBLE ] * initial_susceptible)
    random_nodetypes.extend([ State.INFECTED ] * initial_infected)

    self.random.shuffle(random_nodetypes)

    list_of_random_nodes = self.random.sample(list(self.G), self.agent_count)

    #generates list of random names out of the 200 most common names in the US
    names = generate_names(self.agent_count, self.agent_count*2)
    traits = generate_big5_traits(self.agent_count)

    # Create agents
    i = 0
    for position in list_of_random_nodes:
        agent = Citizen(
                model=self,
                health_condition=random_nodetypes[i],
                name = names[i],
                age = self.random.randrange(18,65),
                traits = traits[i],
                location = 'grid'
            )

        # Add the agent to a random node
        agent.move_to(self.grid[position])
        i+= 1 # Increment i for next iteration

     # Others
    self.running = True
    self.datacollector.collect(self)


  def day_infected_is_4(self):
    return len(self.agents.select(lambda a: a.healing == 2))

  def step(self, epoch):
    # Reset the newly infected nodes for this iteration
    self.newly_infected = []
    # Initialize this epoch's responses dict
    self.epoch = epoch
    self.responses[epoch] = []
    # Call each agent to decide if they are staying home
    self.agents.shuffle_do("prepare_step")
    # Call step function of each agent
    self.agents.shuffle_do("step")
    # Update the states of agents using the newly_infected array
    self.agents.shuffle_do("update_agent_status")
    # Do we add status delta calculation?
    save_graph(epoch, self.grid.G, "graph.json")
    # Collect data
    self.datacollector.collect(self)


  def run(self, n):
    """Run the model for n steps."""
    # end_program = 0
    for epoch in range(n):
      self.step(epoch)
      # check for early stopping
      # if true, break
      # if compute_num_infected(self) == 0:
      #   end_program += 1
      # if end_program == 2:
      #   break


    save_datacollector(self.datacollector.get_model_vars_dataframe())
    save_agent_responses(self.responses)



In [ ]:
class Citizen(CellAgent):
  def __init__(
                self,
                model,
                health_condition, #initial state
                name,
                age,
                traits,
                location
                ):
     super().__init__(model)
     self.state = health_condition
     self.name = name
     self.age = age
     self.location = location
     self.traits = traits
     if health_condition == State.INFECTED:
        self.day_infected = 1
        self.healing = 6
     else:
        self.day_infected = -1
        self.healing = -1


  def get_health_string(self):

    health_strings = [f"{self.name} feels normal.",
                        f"{self.name} has a light cough.",
                        f"{self.name} has a fever and a cough.",
                        ]

    node_state = self.state

    day_infected = 0

    if self.healing != -1:
        day_infected = 6 - self.healing

    if node_state == State.SUSCEPTIBLE or node_state == State.RECOVERED or day_infected < 2:
        return health_strings[0]

    if day_infected == 3 or day_infected == 6:
        return health_strings[1]

    if day_infected == 4 or day_infected == 5:
        return health_strings[2]

  def ask_agent_stay_at_home(self):
    # Used in decide_location method.
    # Returns True or False depending on whether agent wants to stay at home

    reasoning, response = self.get_response_and_reasoning()

    if reasoning is None:
        reasoning = f"{self.name} did not give a reason."
        print("Reasoning was none-type.")

    response = response.lower()
    if "no" in response:
        return False
    elif "yes" in response:
        return True
    else:
        print(f"Response was something unexpected. Defaulting with assuming agent decided to not stay at home.\nResponse was '{response}'")
        return False

  def get_response_and_reasoning(self):
    # Generate propmt accordingly and call the Generative AI model

    question_prompt = f"""[INST]
        You are {self.name}. You are {self.age} years old.

        Your traits are given below:
        {self.traits}

        Your basic bio is below:
        {self.name} lives in the town of Dewberry Hollow. {self.name} likes the town and has friends who also live there. {self.name} has a job and goes to the office for work everyday.

        I will provide {self.name}'s relevant memories here:
        {self.get_health_string()}
        {self.name} knows about the Catasat virus spreading across the country. It is an infectious disease that spreads from human to human contact via an airborne virus. The deadliness of the virus is unknown. Scientists are warning about a potential epidemic.
        {self.name} checks the newspaper and finds that {(self.model.day_infected_is_4()*100)/self.model.agent_count: .1f}% of Dewberry Hollow's population caught new infections of the Catasat virus yesterday.
        {self.name} goes to work to earn money to support {self.name}'s self.

        Based on the provided memories, should {self.name} stay at home for the entire day? Please provide your reasoning.


        The format should be as follow:
        Reasoning: [explanation]
        Response: [Yes or No]

        Example response format:

        Reasoning: {self.name} is tired.
        Response: Yes

        It is important to provide Response in a single word. Pick either Yes or No, both not accepted.
        There should be 1 reasoning and 1 response section. If multiple reasonings exist, combine them into one.[/INST].
        """

    try:
        # print("Prompt:" , question_prompt)
        start = time.time()
        output = get_completion_from_messages(model = self.model.llm_model,
                                              tokenizer = self.model.tokenizer,
                                              user_prompt = question_prompt)
        end = time.time()
        self.model.inference_count += 1
        self.model.inference_time_taken += (end - start)
        # print("Output for node", curr_node, ":", output)
    except Exception as e:
        print(f"{e}\nProgram paused. Retrying after 10s...")
        time.sleep(10)
        output = get_completion_from_messages(model = self.model.llm_model,
                                              tokenizer = self.model.tokenizer,
                                              user_prompt = question_prompt)

    reasoning = ""
    response = ""
    try:
        parts = output.split('\n')
        # Initialize variables to store the extracted values
        reasoning = ""
        response = ""

        # Loop through the parts and assign values to the variables
        for part in parts:
            if part.startswith("Reasoning:"):
                reasoning = part[len("Reasoning: "):].strip()
            elif part.strip().startswith("Response:"):
                response = part.strip()[len("Response: "):].strip()
                # Remove the period at the end of response if it exists
                if response.endswith('.'):
                    response = response[:-1]

        # save_current_agent_response(self, question_prompt, output, reasoning, response)
        # The following code replaces save_current_agent_response function in Crowd implementation
        simulation_data = {
            "Node": self.cell.coordinate,
            "Prompt": question_prompt,
            "Output": output,
            "Reasoning": reasoning,
            "Response": response
        }

        # responses can be a dictionary
        # responses[0][{}]
        self.model.responses[self.model.epoch].append(simulation_data)
    except:
        print("Reasoning or response were not parsed correctly.")
        response = "No"
        reasoning = None
    return reasoning, response


  def decide_location(self):
    # For each node/person/agent decide if staying home or not
    # In the original implementation, it was called for each agent separately in their prepare_step function
    # We don't allow such structure, but this implementation basically does the same thing
    response = self.ask_agent_stay_at_home()

    # Update agent's location wrt the response
    if response is True:
       self.location = "home"
    else:
        self.location = "grid"

  def prepare_step(self):
    '''
      Make all agents decide on their location before the step functions
    '''
    self.decide_location()

  def step(self):
    '''
      Step function for agent
    '''
    if self.location == 'grid' and self.state == State.SUSCEPTIBLE:
      self.interact()

  def interact(self):
    # neighbors try to influence this node
    neighbors = [agent for agent in self.cell.neighborhood.agents if agent is not self]

    graph = self.model.grid.G

    # loop over neighbors
    for v in neighbors:
        if v.state == State.INFECTED:
            #Generate a random number
            rand = self.random.random()

            if rand < 0.1: # 0.1 is the infection probability
                self.model.newly_infected.append(self)
                return

  def update_agent_status(self):
    if self.state == State.RECOVERED:
      return
    elif self.state == State.INFECTED:
      self.healing -= 1
      if self.healing == 0: # healed
        self.healing = -1
        self.day_infected = -1
        self.state = State.RECOVERED
      else:
        self.day_infected += 1
    else: # if state is susceptible
      if self in self.model.newly_infected:
        self.state = State.INFECTED
        self.healing = 6
        self.day_infected = 1




In [ ]:
# generate_names and generate_big5_traits methods directly taken from: GABM-Epidemic
# https://github.com/bear96/GABM-Epidemic/blob/main/utils.py#L18

def generate_names(n: int, s: int, country_alpha2='US'):
    '''
    Returns random names as names for agents from top names in the USA
    Used in World.init to initialize agents
    '''

    # This function will randomly selct n names (n/2 male and n/2 female) without
    # replacement from the s most popular names in the country defined by country_alpha2
    if n % 2 == 1:
        n += 1
    if s % 2 == 1:
        s += 1

    nd = NameDataset()
    male_names = nd.get_top_names(s//2, 'Male', country_alpha2)[country_alpha2]['M']
    female_names = nd.get_top_names(s//2, 'Female', country_alpha2)[country_alpha2]['F']
    if s < n:
        raise ValueError(f"Cannot generate {n} unique names from a list of {s} names.")
    # generate names without repetition
    names = random.sample(male_names, k=n//2) + random.sample(female_names, k=n//2)

    random.shuffle(names)
    return names

In [ ]:
def generate_big5_traits(n: int):
    '''
    Return big 5 traits for each agent
    Used in World.init to initialize agents
    '''

    #Trait generation
    agreeableness_pos=['Cooperation','Amiability','Empathy','Leniency','Courtesy','Generosity','Flexibility',
                        'Modesty','Morality','Warmth','Earthiness','Naturalness']
    agreeableness_neg=['Belligerence','Overcriticalness','Bossiness','Rudeness','Cruelty','Pomposity','Irritability',
                        'Conceit','Stubbornness','Distrust','Selfishness','Callousness']
    #Did not use Surliness, Cunning, Predjudice,Unfriendliness,Volatility, Stinginess

    conscientiousness_pos=['Organization','Efficiency','Dependability','Precision','Persistence','Caution','Punctuality',
                            'Punctuality','Decisiveness','Dignity']
    #Did not use Predictability, Thrift, Conventionality, Logic
    conscientiousness_neg=['Disorganization','Negligence','Inconsistency','Forgetfulness','Recklessness','Aimlessness',
                            'Sloth','Indecisiveness','Frivolity','Nonconformity']

    surgency_pos=['Spirit','Gregariousness','Playfulness','Expressiveness','Spontaneity','Optimism','Candor']
    #Did not use Humor, Self-esteem, Courage, Animation, Assertion, Talkativeness, Energy level, Unrestraint
    surgency_neg=['Pessimism','Lethargy','Passivity','Unaggressiveness','Inhibition','Reserve','Aloofness']
    #Did not use Shyness, Silenece

    emotional_stability_pos=['Placidity','Independence']
    emotional_stability_neg=['Insecurity','Emotionality']
    #Did not use Fear, Instability, Envy, Gullibility, Intrusiveness

    intellect_pos=['Intellectuality','Depth','Insight','Intelligence']
    #Did not use Creativity, Curiousity, Sophistication
    intellect_neg=['Shallowness','Unimaginativeness','Imperceptiveness','Stupidity']


    #Combine each trait
    agreeableness_tot = agreeableness_pos + agreeableness_neg
    conscientiousness_tot = conscientiousness_pos + conscientiousness_neg
    surgency_tot = surgency_pos + surgency_neg
    emotional_stability_tot = emotional_stability_pos + emotional_stability_neg
    intellect_tot = intellect_pos + intellect_neg

    #create traits list to be returned
    traits_list = []

    for _ in range(n):
        agreeableness_rand = random.choice(agreeableness_tot)
        conscientiousness_rand = random.choice(conscientiousness_tot)
        surgency_rand = random.choice(surgency_tot)
        emotional_stability_rand = random.choice(emotional_stability_tot)
        intellect_rand = random.choice(intellect_tot)

        selected_traits=[agreeableness_rand,conscientiousness_rand,surgency_rand,
                                emotional_stability_rand,intellect_rand]

        traits_chosen = (', '.join(selected_traits))
        traits_list.append(traits_chosen)

    return traits_list

In [ ]:
# Define a function to generate response using Hugging Face model
def get_completion_from_messages(model, tokenizer, user_prompt, max_tokens=200, temperature=0.1):
    try:
        # Tokenize the input with padding
        inputs = tokenizer(user_prompt, return_tensors="pt", padding=True, truncation=True).to("cuda")

        # start_time = time.time()  # Start timer

        # Generate text with attention mask and padding token set
        outputs = model.generate(
            inputs.input_ids,
            max_new_tokens=max_tokens,
            temperature=temperature,
            do_sample=True,
            attention_mask=inputs["attention_mask"],
            pad_token_id= tokenizer.eos_token_id,  # Ensure the padding is handled
        )

        # end_time = time.time()  # End timer
        # print(f"Time taken: {end_time - start_time:.6f} seconds in get_completion_from_messages")

        # Remove the input part from the output (post-processing step)
        outputs = outputs[:, inputs.input_ids.shape[-1]:]

        # Decode the generated tokens to return the text
        return tokenizer.decode(outputs[0], skip_special_tokens=True)

    except Exception as e:
        print(f"Error generating text: {e}")
        return None


In [ ]:
def save_graph(epoch_num, current_graph, file_name):
    path = os.path.join("/content/drive/My Drive/Crowd_Related_Work/mesa", file_name)
    data = nx.node_link_data(current_graph, edges="links")

    # Load existing data if the file already exists
    try:
        with open(path, 'r') as f:
            existing_data = json.load(f)
            # print(existing_data)
    except (FileNotFoundError, json.JSONDecodeError):
        existing_data = {}

    # Add new data to the existing data
    existing_data[epoch_num] = data

    # Write back to file
    with open(path, 'w') as f:
        json.dump(existing_data, f, indent=4)

def save_datacollector(data_frame):
    path = os.path.join("/content/drive/My Drive/Crowd_Related_Work/mesa", "datacollector.json")
    data = data_frame.to_json()

    # Write to file
    with open(path, 'w') as f:
        json.dump(data, f, indent=4)

def save_agent_responses(data_dict):
    path = os.path.join("/content/drive/My Drive/Crowd_Related_Work/mesa", "agent_responses.json")

    # Write to file
    with open(path, 'w') as f:
        json.dump(data_dict, f, indent=4)

In [ ]:
# Statistic methods
def compute_num_on_grid(model):
    return len(model.agents.select(lambda a: a.location == 'grid'))

def compute_num_at_home(model):
    return len(model.agents.select(lambda a: a.location == 'home'))

def compute_num_susceptible(model):
    return len(model.agents.select(lambda a: a.state == State.SUSCEPTIBLE))

def compute_num_infected(model):
    return len(model.agents.select(lambda a: a.state == State.INFECTED))

def compute_num_recovered(model):
    return len(model.agents.select(lambda a: a.state == State.RECOVERED))


In [ ]:
def run_model(model):
    """
    Run an experiment with a given model, and plot the results.
    """
    # draw_space(model.grid, agent_portroyal)
    start = time.time()
    model.run(10)
    end = time.time()
    print("Total time to run the simulation: ", end-start, " seconds")
    print(gabm_model.datacollector.get_model_vars_dataframe())

In [ ]:
from huggingface_hub import login
login(token="your_huggingface_token")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "mistralai/Mistral-7B-Instruct-v0.3"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define configuration for 8-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_8bit_fp32_cpu_offload=True
)

# Load the model with quantization and a manual device map
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,  # Use quantization for 8-bit loading
    device_map="auto"  # Automatically allocate layers to devices
)

# Now you can proceed with using the model for inference


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
# # Add a new pad token to the tokenizer
# tokenizer.add_special_tokens({'pad_token': '<pad>'})

# # Resize the token embeddings in the model to accommodate the new token
# model.resize_token_embeddings(len(tokenizer))

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

In [ ]:
# Code to execute starts here.
# Before this cell, all are definitions and imports
gabm_model = World(
    initial_susceptible=8,
    initial_infected=2,
    network_degree=5,
    llm_model=model,
    tokenizer = tokenizer
)

run_model(gabm_model)

Total time to run the simulation:  670.7489213943481  seconds
    Susceptible  Infected  Recovered  # Home  # Grid
0             8         2          0       0      10
1             8         2          0       3       7
2             8         2          0       3       7
3             7         3          0       5       5
4             7         3          0       4       6
5             7         3          0       7       3
6             7         1          2       5       5
7             7         1          2       5       5
8             7         1          2       8       2
9             7         0          3       5       5
10            7         0          3       2       8


In [ ]:
avg_time = gabm_model.inference_time_taken / gabm_model.inference_count
print(avg_time)

6.706137044429779


In [ ]:
print(gabm_model.datacollector.get_model_vars_dataframe())

    Susceptible  Infected  Recovered  # Home  # Grid
0             8         2          0       0      10
1             7         3          0       3       7
2             4         6          0       3       7
3             4         6          0       4       6
4             2         8          0       5       5
5             2         8          0       9       1
6             2         6          2      10       0
7             2         5          3       8       2
8             2         2          6       8       2
9             2         2          6       8       2
10            2         0          8       5       5
11            2         0          8       4       6


In [ ]:
print(gabm_model.agents[0].traits)

Flexibility, Precision, Spirit, Emotionality, Intelligence
